## Reading nexus files (mantid format)
This example looks for data in mantid files, the format is non standard, i.e. no default and signal attributes where found after trying the three alternative algorithms for [locating plottable data](https://manual.nexusformat.org/datarules.html#find-plottable-data).
The final example found some data by looking up the "axis" keyword, which deviates from the nexus documentation.

In [5]:
# read nexus file
# https://manual.nexusformat.org/datarules.html#find-plottable-data
# Explore different options to get to the data on nexus files created by mantid

# library for opening and reading nexus files using HDF5 format
import h5py
# library to get base path for current user
import os

# os.environ['USERPROFILE'] retrieves the base path for current user
# in windows: C:/users/current_user/
filename = os.environ['USERPROFILE'] + '\Desktop\MantisData\TrainingCourseData' + "\LogWS.nxs"
filename = os.environ['USERPROFILE'] + '\Desktop\MantisData\TrainingCourseData' + "\MUSR00015189_cropped.nxs"
# version 1
# 1 Start at the top level of the NeXus data file (the root of the HDF5 hierarchy).
# 2 Pick the default NXentry group.
#   2a If the root has an attribute default, then its value is the name of the NXentry
#      group to be used.
#   2b Otherwise, pick any NXentry group. This is trivial if there is only one NXentry group.
# 3 Pick the default NXdata group.
#   Open the NXentry group selected above. 
#   3a If it has an attribute default, then its value is the name of the NXdata group to be used. 
#   3b Otherwise, pick any NXdata group. This is trivial if there is only one NXdata group.
# 4 Pick the default plottable field (the signal data).
#   Open the NXdata group selected above. 
#   4a If it has an attribute signal, then its value is the name of the field (dataset) to be plotted. 
#   4b If signal attribute is not present on the NXdata group, then proceed to try an older NeXus 
#      method to find the default plottable data
with h5py.File(filename, "r") as nx:
    print(f"file: {nx.filename}")
    signal_found = False
    if "default" in nx.keys():
        print("found default NXentry group")
    else:
        print("default NXentry group not found check each entry group (2b)")
        # get the keys to the groups
        for entry_key in nx.keys():
            print("NXentry group", entry_key, " - ", type(nx[entry_key]))
            nx_entry = nx[entry_key]
            if "default" in nx_entry.keys():
                print("found default NXdata group")
            else:
                print("default NXdata group not found (3b)")
                for data_key in nx_entry.keys():
                    #print (data_key)
                    if type(nx_entry[data_key]) == h5py._hl.group.Group:
                        print("NXdata group", data_key, "-", nx_entry[data_key], type(nx_entry[data_key]))
                        nx_data = nx_entry[data_key]
                        if "signal" in nx_data.keys():
                            print(nx_data.keys())
                            signal_found = True
                        else:
                            print("signal attribute is not present")
                        
    if signal_found:
        print("can use latest method for plotting")
    else:
        print("try an older NeXus method to find the default plottable data")
    

file: C:\Users\scman1\Desktop\MantisData\TrainingCourseData\MUSR00015189_cropped.nxs
default NXentry group not found check each entry group (2b)
NXentry group mantid_workspace_1  -  <class 'h5py._hl.group.Group'>
default NXdata group not found (3b)
NXdata group instrument - <HDF5 group "/mantid_workspace_1/instrument" (7 members)> <class 'h5py._hl.group.Group'>
signal attribute is not present
NXdata group logs - <HDF5 group "/mantid_workspace_1/logs" (32 members)> <class 'h5py._hl.group.Group'>
signal attribute is not present
NXdata group process - <HDF5 group "/mantid_workspace_1/process" (3 members)> <class 'h5py._hl.group.Group'>
signal attribute is not present
NXdata group sample - <HDF5 group "/mantid_workspace_1/sample" (7 members)> <class 'h5py._hl.group.Group'>
signal attribute is not present
NXdata group workspace - <HDF5 group "/mantid_workspace_1/workspace" (4 members)> <class 'h5py._hl.group.Group'>
signal attribute is not present
try an older NeXus method to find the defau

In [6]:
# version 2 axes attribute
# 1 Start at the top level of the NeXus data file (the root of the HDF5 hierarchy).
# 2 Loop through the groups with class NXentry until the next step succeeds.
# 3 Open the NXentry group and loop through the subgroups with class NXdata until the next step succeeds.
# 4 Open the NXdata group and loop through the fields for the one field with attribute signal="1". 
#   Note: There should be only one field that matches.
with h5py.File(filename, "r") as nx:
    print(f"file: {nx.filename}")
    signal_found = False
    for entry_key in nx.keys():
        print("2 NXentry group", entry_key, " - ", type(nx[entry_key]))
        nx_entry = nx[entry_key]
        for data_key in nx_entry.keys():
            if type(nx_entry[data_key]) == h5py._hl.group.Group:
                print("3 NXdata group", data_key, "-", nx_entry[data_key], type(nx_entry[data_key]))
                nx_data = nx_entry[data_key]
                for data_attr in nx_data.keys():
                    if type(nx_data[data_attr]) == h5py._hl.group.Group:
                        print("Group ", data_attr, nx_data[data_attr])
                    else:
                        if "axes" in data_attr:
                                signal_found = true
                        print("other ", data_attr, nx_data[data_attr].name, nx_data[data_attr].shape, nx_data[data_attr].dtype)
    if signal_found:
        print("can use latest method for plotting")
    else:
        print("try an older NeXus method to find the default plottable data")


file: C:\Users\scman1\Desktop\MantisData\TrainingCourseData\MUSR00015189_cropped.nxs
2 NXentry group mantid_workspace_1  -  <class 'h5py._hl.group.Group'>
3 NXdata group instrument - <HDF5 group "/mantid_workspace_1/instrument" (7 members)> <class 'h5py._hl.group.Group'>
Group  detector <HDF5 group "/mantid_workspace_1/instrument/detector" (5 members)>
Group  instrument_parameter_map <HDF5 group "/mantid_workspace_1/instrument/instrument_parameter_map" (5 members)>
other  instrument_source /mantid_workspace_1/instrument/instrument_source (1,) |S19
Group  instrument_xml <HDF5 group "/mantid_workspace_1/instrument/instrument_xml" (3 members)>
other  name /mantid_workspace_1/instrument/name (1,) |S4
Group  physical_detectors <HDF5 group "/mantid_workspace_1/instrument/physical_detectors" (5 members)>
Group  physical_monitors <HDF5 group "/mantid_workspace_1/instrument/physical_monitors" (1 members)>
3 NXdata group logs - <HDF5 group "/mantid_workspace_1/logs" (32 members)> <class 'h5py._h

In [7]:
# version 3 axis attribute
# 1 Start at the top level of the NeXus data file (the root of the HDF5 hierarchy).
# 2 Loop through the groups with class NXentry until the next step succeeds.
# 3 Open the NXentry group and loop through the subgroups with class NXdata until the next step succeeds.
# 4 Open the NXdata group and loop through the fields for the one field with attribute signal="1". 
#   Note: There should be only one field that matches.
with h5py.File(filename, "r") as nx:
    print(f"file: {nx.filename}")
    signal_found = False
    for entry_key in nx.keys():
        print("2 NXentry group", entry_key, " - ", type(nx[entry_key]))
        nx_entry = nx[entry_key]
        for data_key in nx_entry.keys():
            if type(nx_entry[data_key]) == h5py._hl.group.Group:
                print("3 NXdata group", data_key, "-", nx_entry[data_key], type(nx_entry[data_key]))
                nx_data = nx_entry[data_key]
                for data_attr in nx_data.keys():
                    if type(nx_data[data_attr]) == h5py._hl.group.Group:
                        print("Group ", data_attr, nx_data[data_attr], nx_data.attrs)
                    else:
                        if "axis" in data_attr:
                            signal_found = True
                            print("other ", data_attr, nx_data[data_attr].name, nx_data[data_attr].shape, nx_data[data_attr].dtype)
                            print(nx_data[data_attr][()],nx_data[data_attr].value )
                            print(type(nx_data[data_attr].value))
                            
    if signal_found:
        print("found plottable data")
    else:
        print("try an older NeXus method to find the default plottable data")

file: C:\Users\scman1\Desktop\MantisData\TrainingCourseData\MUSR00015189_cropped.nxs
2 NXentry group mantid_workspace_1  -  <class 'h5py._hl.group.Group'>
3 NXdata group instrument - <HDF5 group "/mantid_workspace_1/instrument" (7 members)> <class 'h5py._hl.group.Group'>
Group  detector <HDF5 group "/mantid_workspace_1/instrument/detector" (5 members)> <Attributes of HDF5 object at 109768048>
Group  instrument_parameter_map <HDF5 group "/mantid_workspace_1/instrument/instrument_parameter_map" (5 members)> <Attributes of HDF5 object at 109768048>
Group  instrument_xml <HDF5 group "/mantid_workspace_1/instrument/instrument_xml" (3 members)> <Attributes of HDF5 object at 109768048>
Group  physical_detectors <HDF5 group "/mantid_workspace_1/instrument/physical_detectors" (5 members)> <Attributes of HDF5 object at 109768048>
Group  physical_monitors <HDF5 group "/mantid_workspace_1/instrument/physical_monitors" (1 members)> <Attributes of HDF5 object at 109768048>
3 NXdata group logs - <HDF

C:\Users\scman1\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:24: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
C:\Users\scman1\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:25: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


In [8]:
# recursively loop on all groups until data is found and print its contents

def look_up_group(nx_group):
    for group_key in nx_group.keys():
        #stop condition 
        if type(nx_group[group_key]) != h5py._hl.group.Group:
            print("other ", group_key, nx_group[group_key].name, nx_group[group_key].shape, nx_group[group_key].dtype)
            print(nx_group[group_key][()])
            print(type(nx_group[group_key][()]))
        elif type(nx_group[group_key]) == h5py._hl.group.Group:
            look_up_group(nx_group[group_key])

with h5py.File(filename, "r") as nx:
    print(f"file: {nx.filename}")
    look_up_group(nx)

file: C:\Users\scman1\Desktop\MantisData\TrainingCourseData\MUSR00015189_cropped.nxs
other  definition /mantid_workspace_1/definition (1,) |S27
[b'Mantid Processed Workspace']
<class 'numpy.ndarray'>
other  definition_local /mantid_workspace_1/definition_local (1,) |S27
[b'Mantid Processed Workspace']
<class 'numpy.ndarray'>
other  detector_count /mantid_workspace_1/instrument/detector/detector_count (1,) int32
[1]
<class 'numpy.ndarray'>
other  detector_index /mantid_workspace_1/instrument/detector/detector_index (1,) int32
[0]
<class 'numpy.ndarray'>
other  detector_list /mantid_workspace_1/instrument/detector/detector_list (1,) int32
[2]
<class 'numpy.ndarray'>
other  detector_positions /mantid_workspace_1/instrument/detector/detector_positions (1, 3) float64
[[   0.20155644   43.99491399 -118.125     ]]
<class 'numpy.ndarray'>
other  spectra /mantid_workspace_1/instrument/detector/spectra (1,) int32
[2]
<class 'numpy.ndarray'>
other  author /mantid_workspace_1/instrument/instrument

other  time /mantid_workspace_1/logs/Field_T20/time (10,) float64
[  0.  34.  67. 101. 134. 168. 201. 236. 270. 304.]
<class 'numpy.ndarray'>
other  value /mantid_workspace_1/logs/Field_T20/value (10,) float64
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
<class 'numpy.ndarray'>
other  time /mantid_workspace_1/logs/Field_ZF_Magnitude/time (10,) float64
[  0.  34.  67. 101. 134. 168. 201. 236. 270. 304.]
<class 'numpy.ndarray'>
other  value /mantid_workspace_1/logs/Field_ZF_Magnitude/value (10,) float64
[8.00734997 8.00734997 8.00737    8.00743008 8.00685024 8.00747967
 8.00728989 8.00708961 8.00728989 8.00737   ]
<class 'numpy.ndarray'>
other  time /mantid_workspace_1/logs/ICPdebug/time (84,) float64
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 251. 251. 251. 251.
 251. 251. 251. 251. 251. 251. 251. 251. 251. 251. 251. 251. 251. 251.
 251. 251. 251. 251. 251. 251. 251. 251. 251. 251. 251. 251. 251. 251.
 251. 251. 251. 251. 251. 251. 251. 251. 252. 252. 252. 252. 252. 252.
 252. 252. 252. 252.

other  author /mantid_workspace_1/process/MantidAlgorithm_1/author (1,) |S6
[b'mantid']
<class 'numpy.ndarray'>
other  data /mantid_workspace_1/process/MantidAlgorithm_1/data (1,) |S1193
[b'Algorithm: Load v1\nExecution Date: 2013-Sep-05 14:03:51\nExecution Duration: 0.106 seconds\nParameters:\n  Name: Filename, Value: \\\\olympic\\babylon5\\Public\\mantid\\SampleData\\MUSR00015189.nxs, Default?: No, Direction: Input\n  Name: OutputWorkspace, Value: MUSR00015189, Default?: No, Direction: Output\n  Name: LoaderName, Value: LoadMuonNexus, Default?: No, Direction: Output\n  Name: LoaderVersion, Value: 1, Default?: No, Direction: Output\n  Name: SpectrumMin, Value: 0, Default?: Yes, Direction: Input\n  Name: SpectrumMax, Value: 2147483647, Default?: Yes, Direction: Input\n  Name: SpectrumList, Value: , Default?: Yes, Direction: Input\n  Name: AutoGroup, Value: 0, Default?: Yes, Direction: Input\n  Name: EntryNumber, Value: 0, Default?: Yes, Direction: Input\n  Name: MainFieldDirection, Val